This notebook demonstrates the mechanics of translating an HGVS expression to a VR representation for educational purposes. Users who wish to translate HGVS or other expressions routinely should use ga4gh.vrs.extras.translator.

In [1]:
from ga4gh.core import ga4gh_identify
from ga4gh.vrs import models
from ga4gh.vrs.dataproxy import SeqRepoRESTDataProxy
from ga4gh.vrs.extras.translator import Translator

## Translate an expression manually

First, we'll translate NC_000013.11:g.32936732G>C to VR manually to see the evolution of the process. 

In [2]:
# We'll translate this expression to VR:
hgvs_expr = ""

In [3]:
allele = models.Allele(
    location = models.SequenceLocation(
        sequence_id = "refseq:NC_000013.11",
        interval = models.SimpleInterval(
            start = 32936731,
            end = 32936732
        )
    ),
    state = models.SequenceState(
        sequence = "C"
    )
)

allele.as_dict()

{'location': {'interval': {'end': 32936732,
   'start': 32936731,
   'type': 'SimpleInterval'},
  'sequence_id': 'refseq:NC_000013.11',
  'type': 'SequenceLocation'},
 'state': {'sequence': 'C', 'type': 'SequenceState'},
 'type': 'Allele'}

<div class="alert alert-success">
    👍 The above message is a valid VR message. VR requires that
    sequence identifiers use CURIE syntax with a namespace from identifiers.org and
    that locations are specified with interbase coordinates. Using sequence digests
    is NOT required.
    </div>

## Replace the RefSeq sequence with a GA4GH sequence id

In order to use the computed identifier mechanism in VR, the sequence_id MUST use
GA4GH computed sequence identifiers. 

Implementations choose how to provide sequence and sequence accession services

The following uses the [seqrepo REST interface](https://github.com/biocommons/seqrepo-rest-service/). See the vr-python docker instructions for a simple way to set this up.


In [4]:
seqrepo_rest_service_url = "http://localhost:5000/seqrepo"
dp = SeqRepoRESTDataProxy(base_url=seqrepo_rest_service_url)

# In general, one identifier may be related to many others in another namespace
# Therefore, translate_sequence_identifier() returns a list.
# Because there will be only 1 ga4gh sequence digest, we choose the first
# and then replace the sequence id in allele.location.

refseq_ir = str(allele.location.sequence_id)
ga4gh_ir = dp.translate_sequence_identifier(refseq_ir, "ga4gh")[0]
ga4gh_ir

'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT'

In [5]:
# Now, simply replace the identifier with the GA4GH identifier
allele.location.sequence_id = ga4gh_ir
allele.as_dict()

{'location': {'interval': {'end': 32936732,
   'start': 32936731,
   'type': 'SimpleInterval'},
  'sequence_id': 'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
  'type': 'SequenceLocation'},
 'state': {'sequence': 'C', 'type': 'SequenceState'},
 'type': 'Allele'}

## Computed Identifiers (optional)

ga4gh_identify() serializes the object and computes the identifier
(See ga4gh_serialize and ga4gh_digest for details)


In [6]:
allele._id = ga4gh_identify(allele)
allele.as_dict()

{'_id': 'ga4gh:VA.n9ax-9x6gOC0OEt73VMYqCBfqfxG1XUH',
 'location': {'interval': {'end': 32936732,
   'start': 32936731,
   'type': 'SimpleInterval'},
  'sequence_id': 'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
  'type': 'SequenceLocation'},
 'state': {'sequence': 'C', 'type': 'SequenceState'},
 'type': 'Allele'}

# Using ga4gh.vrs.extras.translator

The VR Translator imports HGVS, SPDI, Beacon, and VCF formats, and appropriate handles more complex cases than shown above.

By default, the translator translates HGVS reference sequences to GA4GH sequence digest identifiers and adds identifiers to the resulting Allele objects.

In [7]:
tlr = Translator(data_proxy=dp)

### HGVS → VR

In [8]:
hgvs_expr1 = "NC_000013.11:g.32936732C>G"
allele1 = tlr.from_hgvs(hgvs_expr1)
allele1.as_dict()

{'_id': 'ga4gh:VA.gvCtR5KLdng5G31DwajXiH6S3Gjhm5fh',
 'location': {'interval': {'end': 32936732,
   'start': 32936731,
   'type': 'SimpleInterval'},
  'sequence_id': 'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
  'type': 'SequenceLocation'},
 'state': {'sequence': 'G', 'type': 'SequenceState'},
 'type': 'Allele'}

### VR → HGVS

Because a GA4GH sequence identifier may have many aliases, each VR Allele
may be expressed as multiple HGVS expressions. For this reason, `to_hgvs()` returns
a *list* of HGVS expressions, optionally limited to aliases from a specified namespace.

In [9]:
tlr.to_hgvs(allele)

['da80c1b72d32295f701e8ee083e06df9f3e08b9a:g.32936732=',
 'GS__0wi-qoDrvram155UmcSC-zA5ZK4fpLT:g.32936732=',
 '2oDBty0yKV9wHo7gg+Bt+fPgi5o:g.32936732=',
 'a5437debe2ef9c9ef8f3ea2874ae1d82:g.32936732=',
 'ff4c22faaa03aefada9b5e795267120becc0e592b87e92d3:g.32936732=',
 'SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT:g.32936732=',
 'NC_000013.11:g.32936732=',
 'CM000675.2:g.32936732=']

In [10]:
# Most commonly, we'll want expressions from a well-known authority like RefSeq
# Again, there might in general be multiple `refseq` expressions
tlr.to_hgvs(allele, "refseq")

['NC_000013.11:g.32936732=']

In [11]:
# GRC namespaces is handled as a special case: Because aliases are shared 
# between GRCh releases, they're shown only on request
tlr.to_hgvs(allele, "GRCh38")

['chr13:g.32936732=', '13:g.32936732=']

In [12]:
# Round-trip test: HGVS → VR Allele → HGVS[]
for hgvs_expr in (
    "NC_000013.11:g.32936732C>G",
    "NC_000013.11:g.32936732_32936733del",
    "NC_000013.11:g.32936732_32936737del",
    "NC_000013.11:g.32936732_32936733insC",
    "NC_000013.11:g.32936732_32936733delinsC",
    "NC_000013.11:g.32936732_32936735delinsC",
):
    a = tlr.from_hgvs(hgvs_expr)
    he = tlr.to_hgvs(a, "refseq")
    chk = "✔" if hgvs_expr in he else "✘"
    print(f"{chk} {hgvs_expr}\n  → {ga4gh_identify(a)}\n  → {he}")

✔ NC_000013.11:g.32936732C>G
  → ga4gh:VA.gvCtR5KLdng5G31DwajXiH6S3Gjhm5fh
  → ['NC_000013.11:g.32936732C>G']
✔ NC_000013.11:g.32936732_32936733del
  → ga4gh:VA.yOoxi7-uUnJyn4QkQ23h6RJuT4Zqarow
  → ['NC_000013.11:g.32936732_32936733del']
✔ NC_000013.11:g.32936732_32936737del
  → ga4gh:VA.nJqbt_W7xV07irZ_F5mtsh5e5dkq9dBW
  → ['NC_000013.11:g.32936732_32936737del']
✔ NC_000013.11:g.32936732_32936733insC
  → ga4gh:VA.JEUN0DVx2gySgRhNDqlKYqehZxgKKlsY
  → ['NC_000013.11:g.32936732_32936733insC']
✔ NC_000013.11:g.32936732_32936733delinsC
  → ga4gh:VA.cT0SNJb9bxB_KIhu2s6j37ZbTWaU4ozJ
  → ['NC_000013.11:g.32936732_32936733delinsC']
✔ NC_000013.11:g.32936732_32936735delinsC
  → ga4gh:VA.6ZgsF2lSBqMKcGL-xV-SUSrwN_UQTndJ
  → ['NC_000013.11:g.32936732_32936735delinsC']
